In [1]:
import pandas as pd
import numpy as np

# input data 부분 필요 라이브러리
import kqc_custom
from dataclasses import dataclass, field, asdict, astuple
from typing import List, ClassVar
import ast  # 입력 문자 beta coef, integer 로 변환에 사용하는 부분

# Simulated Annealing 필요 라이브러리
from sa_basefunctions import get_aic, get_bic, get_mspe, get_bin, flip
from sa_SimulatedAnnealing import SimulatedAnnealing

# Genetic Algorithm 필요 라이브러리 

## Data Generator 구현 부분

*Data Generating Function 함수 설명*

$y = X \beta + \epsilon$

kqc_custom.generate_independent_sample(n_sample,n_features,beta_coef=[4,3,2,2],epsilon=4)  
kqc_custom.generate_dependent_sample(n_sample,n_features,beta_coef=[4,3,2,2],epsilon=4,correlation_parameter=1)

- n_samples : 데이터 샘플 수
- n_features : 데이터 피쳐 개수
- beta_coef : beta계수
- epsilon : 오차의 표준편차 정도
- correlation_parameter


In [2]:
# inout parameter 가져오기
KQC_data_parameter = pd.read_excel('input_parameter.xlsx')

Example of input data generator funvtion's parameter input from EXCEL (.xlcs file)

    아래에 간단한 예시로 4개의 데이터의 경우를 예시로 들었습니다.

In [3]:
KQC_data_parameter

,지정 데이터명,n_samples,n_features,beta_coef,epsilon,correlation,is_independent
0,data1,500,10,"[4,3,2,2]",1,1,independent
1,data2,500,10,"[4,3,2,2]",2,1,independent
2,data3,500,10,"[4,3,2,2]",3,1,independent
3,data4,500,10,"[4,3,2,2]",4,1,independent


In [4]:
def read_excel2tuple():
    '''
        row 별로 data를 슬라이싱 해주는 Tuple로 변환
    '''
    df = pd.read_excel('input_parameter.xlsx')
    return list(df.itertuples(name=None))

In [5]:
Tupled_df_KQC_data_parameter = list(KQC_data_parameter.itertuples(name=None))

In [6]:
list(list(KQC_data_parameter.itertuples(name=None))[0])

[0, 'data1', 500, 10, '[4,3,2,2]', 1, 1, 'independent']

EXCEL 의 dataframee 의 pandas로 받아서, 

In [7]:
from typing import ClassVar
@dataclass
class data_generator:
    count_data : ClassVar[int] = 1  # 전치 처리위한 class variable

    # KQC custom file // data generator function's parameter
    n_samples : int                 # 데이터 샘플 수
    n_features : int                # 데이터 피쳐 개수
    beta_coef : float               # beta 계수
    epsilon : float                 # 오차의 표준편차 정도
    correlation_parameter : float   # 상관계수 ?

    # kqc data generator dependent / inpependent 선택 부분 
    is_independent : str            # is_independent
    data_name : str                 # 지정 데이터명

    def __post_init__(self):
        data_generator.count_data += 1  # class 생성 data counter
      
    # parameter 입력 받기 
    @classmethod
    def create(cls): 
        data = cls() 
        return data
    
    @classmethod
    def data_counter(cls):
        return [ 'data' + str(i) for i in range(cls.count_data)]

    #KQC_custom 의 data generator 로 부터 data 생성 class
    def data_gen(self):
        data_gened = []
        if self.is_independent == 'dependent':
            x, y = kqc_custom.generate_dependent_sample( self.n_samples,
                                                  self.n_features,
                                                  self.beta_coef,
                                                  self.epsilon,
                                                  self.correlation_parameter
            )
        else:
            x, y = kqc_custom.generate_independent_sample( self.n_samples,
                                                  self.n_features,
                                                  self.beta_coef,
                                                  self.epsilon,
                                                  self.correlation_parameter
            )
        data_gened.append(x)
        data_gened.append(y)
        return data_gened
# data_name : str 
# paeameter : List[float] = field(default_factory=list)

In [8]:
ast.literal_eval(list(list(KQC_data_parameter.itertuples(name=None))[0])[4])

[4, 3, 2, 2]

In [9]:
# class data generator 로 부터, 입력 데이터 리스트의 수 만큼 리스트 생성 
def list_gen_from_class(df):
    generated_data_list = []
    Tupled_df_KQC_data_parameter = read_excel2tuple()
    for i in range(len(Tupled_df_KQC_data_parameter)):
        data_list_Row = list(list(df.itertuples(name=None))[i])
        generated_data_list.append(data_generator(n_samples= data_list_Row[2],
                                                n_features= data_list_Row[3],
                                                beta_coef= ast.literal_eval(data_list_Row[4]),
                                                epsilon= data_list_Row[5],
                                                correlation_parameter= data_list_Row[6],
                                                is_independent= data_list_Row[7],
                                                data_name=data_list_Row[1]
            )
        )
    return generated_data_list

다음은 generated data list 의 형태를 보여주는 예시입니다.  

아래의 예시와 같이 각 리스트에서 KQC_ independent / dependent data 의 생성되는  
x , y 의 값이 N(excel input 데이터수) by 2 의 size 리스트 입니다.  

In [10]:
generated_data_list = list_gen_from_class(KQC_data_parameter)
#generated_data 의 length --> 클래스 변수에서 선언횟수 카운터로 선언했으므로 
# 임의의 첫번쨰 선언 instance 에서 클래스 매서드를 호출하면 조정없이 가능
data_key = generated_data_list[0].data_counter()

리스트와 동일하지만 검색에 유용하기 위해서, 데이터 명으로 검색을 하기 위해서 딕셔너리로 변환하는  부분을 작성해 보았습니다.  

In [11]:
generated_data_dict = {}
for i in range(len(generated_data_list)):
    '''
        클래스에서 선언 인스턴스 개수로 자동 카운트 되는 이름명은 윗 코드
        dataframe 에서 선언한 '지정 데이터명'을 사용한다면 아래 코드를 사용하면 됩니다.
    '''
    generated_data_dict[data_key[i]] = generated_data_list[i]
    # list(KQC_data_parameter.loc[:,'지정 데이터명'])[i] = generated_data_list[i]

In [12]:
generated_data_dict['data3']

data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=4, correlation_parameter=1, is_independent='independent', data_name='data4')

In [13]:
# 클래스 구조를 가짐으로서 data 이름을 검색한것으로 부터 parameter를 쉽게 확인할 수 있습니다.
generated_data_dict['data3'].__dict__

{'n_samples': 500,
 'n_features': 10,
 'beta_coef': [4, 3, 2, 2],
 'epsilon': 4,
 'correlation_parameter': 1,
 'is_independent': 'independent',
 'data_name': 'data4'}

In [14]:
# 저장된 클래스 인스턴스로 부터 데이터 생성하기 
generated_data_dict['data3'].data_gen()

[array([[ 1.62434536, -0.61175641, -0.52817175, ..., -0.7612069 ,
          0.3190391 , -0.24937038],
        [ 1.46210794, -2.06014071, -0.3224172 , ..., -0.87785842,
          0.04221375,  0.58281521],
        [-1.10061918,  1.14472371,  0.90159072, ..., -0.93576943,
         -0.26788808,  0.53035547],
        ...,
        [-2.01516818, -0.99690494, -1.03425079, ..., -1.16362557,
          0.06157423, -0.57646002],
        [-1.5929365 ,  0.71568559,  0.9248516 , ..., -0.38246272,
          0.97307106,  1.06971394],
        [-0.54441474, -0.06307104,  0.89381886, ..., -0.77598779,
          1.08488869,  2.24198946]]),
 array([-2.23918967e+00,  2.77062608e+00, -2.67530053e+00, -9.92061076e+00,
         9.54648348e-01,  2.91207576e+00,  2.26725404e+00, -7.23531689e+00,
         4.12832328e+00,  6.09632648e+00,  5.34863852e+00, -1.07366807e+01,
         4.52824111e+00, -7.11761308e+00, -4.09005398e+00, -3.29252435e+00,
         2.54994490e+00,  7.94742650e+00, -3.94756430e-01,  1.2017236

## 알고리즘 적용 부분

### GA



In [15]:
from ga_algorithm import GA
from ga_results import ga_Results_gen, ga_Results

SA 와 별개로 ga는 실험 데이터별로 사용할 매개변수의 조정이 별도로 없어서 4가지 종류의 input data로 부터
바로 GA 알고리즘을 실행합니다.  
아래는 생성한 data 4개의 딕셔너리 입니다.

In [16]:
print(generated_data_dict)
len(generated_data_dict.keys())

a,b = list(generated_data_dict.items())[1]
x = pd.DataFrame(b.data_gen()[0])
y = b.data_gen()[1]

print(x)
print(y)
# GA(x, y)

{'data0': data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=1, correlation_parameter=1, is_independent='independent', data_name='data1'), 'data1': data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=2, correlation_parameter=1, is_independent='independent', data_name='data2'), 'data2': data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=3, correlation_parameter=1, is_independent='independent', data_name='data3'), 'data3': data_generator(n_samples=500, n_features=10, beta_coef=[4, 3, 2, 2], epsilon=4, correlation_parameter=1, is_independent='independent', data_name='data4')}
            0         1         2         3         4         5         6  \
0    1.624345 -0.611756 -0.528172 -1.072969  0.865408 -2.301539  1.744812   
1    1.462108 -2.060141 -0.322417 -0.384054  1.133769 -1.099891 -0.172428   
2   -1.100619  1.144724  0.901591  0.502494  0.900856 -0.683728 -0.122890   
3   -0.691661 -0.396754 -0.687173 -

In [17]:
ga_resutls_dict = {}

for data_name, data in generated_data_dict.items():
    x, y = pd.DataFrame(data.data_gen()[0]), data.data_gen()[1]
    ga_best_score, ga_best_solution, ga_result_log = GA(x,y)
    new_name = data_name + '_with_' + 'GA'
    ga_resutls_dict[new_name] = ga_Results(ga_best_score, ga_best_solution, ga_result_log)

### 최종결과 확인 

비고  
4개의 데이터를 input_paramer.xlcs 에서 생성해서 데이터를 만듬  
EXCEl 에서 수정을 통해 원하는데이터 개수만큼 생성 가능 

In [18]:
'''
    ga_results_dict 라는 dictionary 에 input 데이터만큼 저장이 됩니다.
    딕셔너리의 키 인 데이터 이름을 추출해 줍니다. 
'''
ga_result_keys = list(ga_resutls_dict.keys())
ga_result_keys

['data0_with_GA', 'data1_with_GA', 'data2_with_GA', 'data3_with_GA']

In [27]:
print(ga_result_keys[1])
ga_reuslt_data = ga_resutls_dict[ga_result_keys[1]]
#이제 데이터의 3가지 값을 아래와 같이 . 변수명 으로 접근해서 실험값을 추출할 수 있습니다.

data1_with_GA


In [20]:
ga_reuslt_data.best_score

1055.7359313005538

In [21]:
ga_reuslt_data.best_solution

array([1., 1., 0., 1., 1., 1., 0., 1., 0., 0.])

In [22]:
ga_reuslt_data.result_log[1]

'[1. 1. 0. 1. 1. 1. 0. 0. 0. 1.], [1. 1. 0. 1. 1. 1. 1. 0. 0. 1.], [1. 1. 0. 0. 1. 1. 0. 0. 1. 0.], [1. 1. 0. 0. 1. 0. 1. 1. 1. 0.], [1. 1. 0. 0. 0. 0. 0. 1. 0. 1.], [1. 0. 1. 0. 1. 1. 0. 0. 0. 0.], [1. 0. 1. 0. 0. 0. 0. 0. 0. 1.], [1. 0. 0. 1. 1. 0. 1. 0. 0. 1.], [1. 0. 0. 1. 0. 1. 1. 1. 0. 1.], [1. 0. 0. 1. 1. 0. 0. 1. 1. 1.], [1. 0. 0. 0. 1. 0. 1. 1. 1. 0.], [1. 1. 0. 0. 1. 1. 0. 1. 1. 1.], [1. 0. 1. 0. 1. 1. 0. 0. 0. 0.], [0. 1. 0. 1. 0. 0. 0. 0. 1. 0.], [0. 1. 0. 1. 1. 1. 1. 1. 0. 1.], [0. 1. 1. 0. 0. 1. 1. 0. 1. 1.], [1. 0. 0. 0. 1. 1. 0. 1. 1. 1.], [0. 0. 1. 1. 0. 1. 1. 0. 0. 1.], [1. 1. 1. 0. 1. 1. 1. 0. 0. 1.], [1. 0. 1. 0. 0. 0. 0. 0. 0. 1.]'